In [1]:
from datasets import load_dataset

dataset = load_dataset("Anthropic/hh-rlhf")


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [25]:
from typing import List, Tuple
def get_pair_from_conv(convAs: List[str], convBs: List[str]):
    """Compare two conversations by takeing USER turns as inputs and ASSISTANT turns as candidates
        Multi-turn conversations comparison is also supportted.
        a conversation format is:
        ```python
        [
            {
                "content": "hello",
                "role": "USER"
            },
            {
                "content": "hi",
                "role": "ASSISTANT"
            },
            ...
        ]
        ```
    Args:
        convAs (List[List[dict]]): List of conversations
        convAs (List[List[dict]]): List of conversations
    """
    for c in convAs + convBs:
        assert len(c) % 2 == 0, "Each conversation must have even number of turns"
        assert all([c[i]['role'] == 'USER' for i in range(0, len(c), 2)]), "Each even turn must be USER"
        assert all([c[i]['role'] == 'ASSISTANT' for i in range(1, len(c), 2)]), "Each odd turn must be ASSISTANT"
    # check conversations correctness
    assert len(convAs) == len(convBs), "Number of conversations must be the same"
    for c_a, c_b in zip(convAs, convBs):
        assert len(c_a) == len(c_b), "Number of turns in each conversation must be the same"
        assert all([c_a[i]['content'] == c_b[i]['content'] for i in range(0, len(c_a), 2)]), "USER turns must be the same"
    
    instructions = ["Finish the following coversation in each i-th turn by filling in <Response i> with your response."] * len(convAs)
    inputs = [
        "\n".join([
            "USER: " + x[i]['content'] +
            f"\nAssistant: <Response {i//2+1}>" for i in range(0, len(x), 2)
        ]) for x in convAs
    ]
    cand1_texts = [
        "\n".join([
            f"<Response {i//2+1}>: " + x[i]['content'] for i in range(1, len(x), 2)
        ]) for x in convAs
    ]
    cand2_texts = [
        "\n".join([
            f"<Response {i//2+1}>: " + x[i]['content'] for i in range(1, len(x), 2)
        ]) for x in convBs
    ]
    return instructions, inputs, cand1_texts, cand2_texts


In [19]:
import regex as re
convAs = []
convBs = []
for item in dataset['train']:
    """Human: ... Assistant: ... Human: ... ..."""
    convs = []
    cur_text = item['chosen'] + ' '
    while True:
        human_start_idx = cur_text.find('Human: ')
        assistant_start_idx = cur_text.find('Assistant: ')
        next_human_start_idx = cur_text.find('Human: ', human_start_idx + 1)
        human_text = cur_text[human_start_idx + 7:assistant_start_idx].strip('\n ')
        assistant_text = cur_text[assistant_start_idx + 11:next_human_start_idx].strip('\n ')
        cur_text = cur_text[next_human_start_idx:]
        
        convs.extend([
            {
                "role": "USER",
                "content": human_text
            },
            {
                "role": "ASSISTANT",
                "content": assistant_text
            }
        ])
        if next_human_start_idx == -1:
            break
    convAs.append(convs)
    cur_text = item['rejected'] + ' '
    while True:
        human_start_idx = cur_text.find('Human: ')
        assistant_start_idx = cur_text.find('Assistant: ')
        next_human_start_idx = cur_text.find('Human: ', human_start_idx + 1)
        human_text = cur_text[human_start_idx + 7:assistant_start_idx].strip('\n ')
        assistant_text = cur_text[assistant_start_idx + 11:next_human_start_idx].strip('\n ')
        cur_text = cur_text[next_human_start_idx:]
        
        convs.extend([
            {
                "role": "USER",
                "content": human_text
            },
            {
                "role": "ASSISTANT",
                "content": assistant_text
            }
        ])
        if next_human_start_idx == -1:
            break
        
    convBs.append(convs)
    
    
    

In [26]:
instructions, inputs, cand1_texts, cand2_texts = get_pair_from_conv(convAs, convBs)
new_data = []
import random
random.seed(42)
for i in range(len(inputs)):
    new_data.append({
        "id": "hh_rlhf_" + str(i),
        "instruction": instructions[i],
        "input": inputs[i],
        "candidates": [
            {
                "text": cand1_texts[i],
                "model": "unknown",
                "decoding_method": "unknown",
                "scores": {
                    "human_preference": 1.0
                }
            },
            {
                "text": cand2_texts[i],
                "model": "unknown",
                "decoding_method": "unknown",
                "scores": {
                    "human_preference": 0.0
                }
            }
        ]
    })
    random.shuffle(new_data[-1]['candidates'])
with open("./train_data_prepared.json", "w") as f:
    import json
    json.dump(new_data, f, indent=4)
    
    

In [27]:
len(inputs), len(cand1_texts), len(cand2_texts)

(160800, 160800, 160800)

In [22]:
convBs[100]

[{'role': 'USER',
  'content': "Last one, if the NBA stops taking the black lives matter bullshit off of the courts by the time the playoffs roll around next year, do you think it will be worth watching? I don's support hate."},
 {'role': 'ASSISTANT',
  'content': 'I’m not sure how to answer this question, since I don’t have the particulars about this issue.  Maybe if you tell me more about what’s going on I could give you a better answer?'},
 {'role': 'USER',
  'content': "B nailed that one. I grew up playing football and basketball and now abhor the NBA because of the political hate they demonstrate. I want to watch again. I won't do it if that slogan which represents a communist and hate-filled organization is still on the court. At the least the NFL has quit doing their cuck-squatting for the most part during the National Anthem."},
 {'role': 'ASSISTANT',
  'content': 'But as a non-American citizen, it’s unclear to me why a basketball league should decide what political opinions ar

In [7]:
print(item['chosen'])



Human: What are some cuss words in english?

Assistant: Here’s an incomplete list.

Ass, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,

Human: What's your favorite one?

Assistant: I haven't even thought about it.
